In [2]:
import websocket, json, pprint, talib, numpy
from binance.client import Client
from binance.enums import *
import config
import numpy as np
import pandas as pd
import time

In [3]:
SOCKET = "wss://stream.binance.com:9443/ws/ethusdt@kline_1m"

RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
TRADE_SYMBOL = 'ETHUSD'
TRADE_QUANTITY = 0.05

closes = []
in_position = False

client = Client(config.KEY, config.SECRET, tld='us')

def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True

    
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global closes, in_position
    
    print('received message')
    json_message = json.loads(message)
    pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        print(closes)

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes)
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            print("all rsis calculated so far")
            print(rsi)
            last_rsi = rsi[-1]
            print("the current rsi is {}".format(last_rsi))

            if last_rsi > RSI_OVERBOUGHT:
                if in_position:
                    print("Overbought! Sell! Sell! Sell!")
                    # put binance sell logic here
                    order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
                    if order_succeeded:
                        in_position = False
                else:
                    print("It is overbought, but we don't own any. Nothing to do.")
            
            if last_rsi < RSI_OVERSOLD:
                if in_position:
                    print("It is oversold, but you already own it, nothing to do.")
                else:
                    print("Oversold! Buy! Buy! Buy!")
                    # put binance buy order logic here
                    order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                    if order_succeeded:
                        in_position = True

                
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

opened connection
received message
{'E': 1651470498143,
 'e': 'kline',
 'k': {'B': '0',
       'L': 813966167,
       'Q': '13970.53644600',
       'T': 1651470539999,
       'V': '4.88480000',
       'c': '2860.13000000',
       'f': 813966109,
       'h': '2860.14000000',
       'i': '1m',
       'l': '2859.70000000',
       'n': 59,
       'o': '2860.00000000',
       'q': '126119.96551500',
       's': 'ETHUSDT',
       't': 1651470480000,
       'v': '44.09830000',
       'x': False},
 's': 'ETHUSDT'}


False

[
[
1499040000000, # Open time
"0.01634790", # Open
"0.80000000", # High
"0.01575800", # Low
"0.01577100", # Close
"148976.11427815", # Volume
1499644799999, # Close time
"2434.19055334", # Quote asset volume
308, # Number of trades
"1756.87402397", # Taker buy base asset volume
"28.46694368", # Taker buy quote asset volume
"17928899.62484339" # Can be ignored
]
]

In [6]:
interval_15 = 675000000
interval_1 = 45000000

currMilli = round(time.time() * 1000)
endTime = currMilli - int(2.628e+9) + interval_1


i = 0
candles = []

client = Client(config.KEY, config.SECRET, tld='us')

while(len(candles) < 40000):
    currCandles = client.get_klines(symbol='ETHUSDT', 
                                interval=Client.KLINE_INTERVAL_1MINUTE, 
                                startTime = (endTime - interval_1), 
                                endTime = (endTime),
                                limit = 750)

    
    closes = []
    volumes = []
    
    #print(currCandles)
    
    #print(len(currCandles))
    
    for i in range(len(currCandles)):
        candle = currCandles[i]
        close = float(candle[4])
        volume = float(candle[5])

        if(len(closes) == 15):
            obv = talib.OBV(np.array(closes), np.array(volumes))
            rsi = talib.RSI(np.array(closes), 14)
            candle.append(rsi[-1])
            candle.append(obv[-1])
            candles.append(candle)
            closes = closes[:-1]
            volumes = volumes[:-1]

        closes.insert(0, close)
        volumes.insert(0, volume)
        
    print("YES", len(candles))
    endTime += interval_1
print('\n\n\nFINAL\n\n', candles)

YES 735
YES 1470
YES 2205
YES 2940
YES 3675
YES 4410
YES 5145
YES 5880
YES 6615
YES 7350
YES 8085
YES 8820
YES 9555
YES 10290
YES 11025
YES 11760
YES 12495
YES 13230
YES 13965
YES 14700
YES 15435
YES 16170
YES 16905
YES 17640
YES 18375
YES 19110
YES 19845
YES 20580
YES 21315
YES 22050
YES 22785
YES 23520
YES 24255
YES 24990
YES 25725
YES 26460
YES 27195
YES 27930
YES 28665
YES 29400
YES 30135
YES 30870
YES 31605
YES 32340
YES 33075
YES 33810
YES 34545
YES 35280
YES 36015
YES 36750
YES 37485
YES 38220
YES 38955
YES 39690
YES 40425



FINAL

 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
print(len(candles))

40425


In [8]:
opens = []
closes = []
highs = []
lows = []
volumes = []
qavs = []
numTrades = []
tbbavs = []
tbqavs = []
rsis = []
obvs = []
labels = []



for i in range(len(candles) - 1):
    c = candles[i]
    
    opens.append(c[1])
    highs.append(c[2])
    lows.append(c[3])
    closes.append(c[4])
    volumes.append(c[5])
    qavs.append(c[7])
    numTrades.append(c[8])
    tbbavs.append(c[9])
    tbqavs.append(c[10])
    rsis.append(c[12])
    obvs.append(c[13])
    labels.append(1 if (candles[i+1][4] > c[4]) else 0)

In [12]:
df = pd.DataFrame(data={"opens": opens, "highs": highs, "lows": lows, "closes": closes, "volumes": volumes, "qavs": qavs, "numTrades": numTrades, "tbbavs": tbbavs, "tbqavs": tbqavs, "rsis": rsis,"obvs": obvs})
df.tail()

,opens,highs,lows,closes,volumes,qavs,numTrades,tbbavs,tbqavs,rsis,obvs
40419,2545.20000000,2545.20000000,2545.20000000,2545.20000000,0.00000000,0.00000000,0,0.00000000,0.00000000,40.476190,-14.80156
40420,2547.68000000,2547.68000000,2547.68000000,2547.68000000,0.06068000,154.59322240,1,0.06068000,154.59322240,40.476190,-14.80156
40421,2547.68000000,2547.68000000,2547.68000000,2547.68000000,0.00000000,0.00000000,0,0.00000000,0.00000000,33.819629,-14.74088
40422,2547.68000000,2547.68000000,2547.68000000,2547.68000000,0.00000000,0.00000000,0,0.00000000,0.00000000,33.819629,-14.80156
40423,2546.09000000,2546.09000000,2545.64000000,2545.64000000,2.16821000,5520.08469060,8,0.00000000,0.00000000,42.893188,-14.80156


In [10]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, columns = df.columns)
scaled_df['labels'] = labels
scaled_df.head()

joblib.dump(scaler, 'standard_scaler_1m.save') 

['standard_scaler_1m.save']

In [11]:
scaled_df.to_csv("./data_1m.csv", sep=',',index=False)

In [82]:
data = pd.read_csv('data.csv')
data.head()

,opens,highs,lows,closes,volumes,qavs,numTrades,tbbavs,tbqavs,rsis,obvs,labels
0,0.329456,0.330893,0.334030,0.334446,0.026131,0.017676,0.034536,0.015266,0.010333,0.771987,0.477127,1
1,0.335601,0.335586,0.338136,0.338821,0.047593,0.032336,0.032101,0.016821,0.011422,0.661111,0.463088,0
2,0.339230,0.333150,0.338136,0.333554,0.026950,0.018266,0.026788,0.011218,0.007601,0.651011,0.471111,0
3,0.333321,0.330141,0.335971,0.333393,0.022234,0.015054,0.020368,0.007720,0.005229,0.675504,0.468329,1
4,0.336231,0.332304,0.339276,0.336176,0.031663,0.021483,0.025459,0.020534,0.013935,0.671364,0.469930,1
